In [8]:
import csv
from bs4 import BeautifulSoup
import glob

# get all pages from https://www.blocket.se/bilar/sok?filter={"key":"make","values":["Kia"]}&filter={"key":"models","values":["Optima"]}&filter={"key":"modelYear","range":{"start":"2018","end":"2019"}}&filter={"key":"milage","range":{"start":"10000","end":"15000"}}&filter={"key":"fuel","values":["Miljöbränsle/Hybrid"]}

# Combine HTML content from multiple files
html_files = glob.glob('./data/blocket.data*')
html_content = ''

for file_name in html_files:
    with open(file_name, 'r', encoding='utf-8') as file:
        html_content += file.read()

In [9]:
soup = BeautifulSoup(html_content, 'html.parser')

# Define the structure of the CSV
csv_file = open('car_listings.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Title', 'Model Year', 'Price', 'Mileage'])
# Define the structure of the CSV
unique_entries = set()
rows = []

# Parse the HTML to find car listings
for listing in soup.select('a[href*="/annons/"]'):
    try:
        title = listing.select_one('h2').get_text(strip=True)
        details = listing.select('div[class^="TextCallout2__TextCallout2Wrapper-sc-1bir8f0-0 hXtpYd sc-59286bbc-8 jFWlzF !text-xs md:!text-[15px]"]')

        model_year = details[0].get_text(strip=True)
        fuel_type = details[1].get_text(strip=True)
        mileage = details[5].get_text(strip=True)
        transmission = details[3].get_text(strip=True)

        price = listing.select_one('div[data-cy="price"] div').get_text(strip=True)

        # Create a tuple for the entry
        entry = (title, model_year, price, mileage)

        # Check for duplicates
        if entry not in unique_entries:
            unique_entries.add(entry)
            rows.append(entry)
    except Exception as e:
        print(f"Error processing listing: {e}")

with open('car_listings.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Title', 'Model Year', 'Price', 'Mileage'])
    csv_writer.writerows(rows)

print("CSV file created successfully.")


CSV file created successfully.
